Covid-19 Data analysis for Austria

What we will do today:
* Pie plot
* Bar plot
* plotting on a map
* Line plot

The data is taken from here https://github.com/statistikat/coronaDAT.git

The map data for Austria is taken from here: https://www.data.gv.at/katalog/dataset/stat_gliederung-osterreichs-in-gemeinden14f53

The additional info about the districts is takenfrom wikipedia: https://de.wikipedia.org/wiki/Liste_der_Bezirke_und_Statutarst%C3%A4dte_in_%C3%96sterreich


The official data was finally published and can be found here: https://www.data.gv.at/covid-19/

In [0]:
from IPython.display import clear_output

Install the libraries geopandas and gitpython

In [0]:
!pip install geopandas
!pip install gitpython
clear_output()

Import all needed libraries

In [0]:
import geopandas
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import urllib.request
import pathlib

from zipfile import ZipFile
import git
import os

% matplotlib inline

Clone the Git Repository which contains the Covid Data

The data was originally scraped from https://info.gesundheitsministerium.at/ before the layout of the dashboard was completely changed. Meanwhile the data is officially available here: https://www.data.gv.at/covid-19/

In [0]:
os.mkdir("/content/covid-data")

repo = git.Repo.clone_from("https://github.com/statistikat/coronaDAT.git", "/content/covid-data", no_checkout=True)
repo.git.checkout("8f9ffd3b1159172a331bd42ed08a5d2afd24cd8d")

Have a look at the downloaded data. How is the structure? What kind of data does it contain?

Start with [PiePlot](https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.axes.Axes.pie.html) of gender data


In [0]:
gender = pd.read_csv("/content/covid-data/latest/geschlecht.csv", sep=";")

In [0]:
gender.index = gender['geschlecht']

In [0]:
gender

In [0]:
# use the .plot.pie() function
plot = gender.plot.pie(y='freq', figsize=(5, 5), autopct='%1.0f%%', pctdistance=0.3, labeldistance=1.2)

[PiePlot](https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.axes.Axes.pie.html) or [BarPlot](https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.axes.Axes.bar.html#matplotlib.axes.Axes.bar) for age data?

In [0]:
age = pd.read_csv("/content/covid-data/latest/alter.csv", sep=";")

In [0]:
age.index = age['altersgruppe']

In [0]:
age

In [0]:
# use the .plot.pie() function


In [0]:
# use the .plot.bar() function


Show the data for the single districts on a map

In [0]:
cov_district = pd.read_csv("/content/covid-data/latest/bezirke.csv", sep=";")

In [0]:
cov_district.head()

Download the Shapefile for Austria. It's a file that describes the borders of states and villages in Austria.

In [0]:
#https://www.data.gv.at/katalog/dataset/stat_gliederung-osterreichs-in-gemeinden14f53
url = "http://data.statistik.gv.at/data/OGDEXT_GEM_1_STATISTIK_AUSTRIA_20200101.zip"
urllib.request.urlretrieve(url, '/content/Shapefile.zip')

In [0]:
# Create a ZipFile Object and load sample.zip in it
with ZipFile('/content/Shapefile.zip', 'r') as zipObj:
   # Extract all the contents of zip file in different directory
   zipObj.extractall('Shapefile')

In [0]:
# load the .shp file into the variable "austria"
austria = geopandas.read_file('/content/Shapefile/STATISTIK_AUSTRIA_GEM_20200101Polygon.shp', encoding='utf-8')

Look at the shapefile dataframe, plot it

In [0]:
austria.head()

In [0]:
austria.plot()

In [0]:
austria[austria.name == "Linz"].plot()

Load additional Data about the districts (we's especially like to know the number of inhabitants)

In [0]:
git.Git("/content/").clone("https://github.com/female-coders-linz/covid-19-analysis.git")

In [0]:
districts = pd.read_csv("/content/covid-19-analysis/data/Wohnbezirke.csv")

In [0]:
districts.columns = ['district_id', 'district_name', 'bundesland', 'Kfz_Kz', 'area', 'inhabitants', 'population_denseness', 'amount', 'region_name']

In [0]:
districts.head()

In [0]:
def to_numeric(series):
  series = pd.to_numeric(series.apply(lambda x : x.replace(".", "")))
  return series


def clean_districts(districts_df):
  districts_df = districts_df.drop(21)
  districts_df = districts_df.drop(columns=['Kfz_Kz', 'area', 'population_denseness', 'amount', 'region_name'])

  districts_df["inhabitants"] = to_numeric(districts_df["inhabitants"])

  return districts_df

In [0]:
districts = clean_districts(districts)

In [0]:
districts

In [0]:
sum(districts['inhabitants'])

assign district data zu each entry in shapefile

In [0]:
austria.head()

In [0]:
districts.head()

In [0]:
#iterate through the austria dataframe. for each entry find the according district.
#special case vienna: we only have the district data for whole vienna.

dist = []

for idx, row in austria.iterrows():
  for dist_idx, dist_row in districts.iterrows():
    if row['id'].startswith(dist_row['district_id']) and len(dist) == idx:
      dist.append(dist_row['district_id'])
  if row['id'].startswith('9') and len(dist) == idx:
    dist.append('900')
  #if len(bzk) == idx:
    #bzk.append('0')
    #print(row['id'])
    #print(row['name'])


In [0]:
austria['district_id'] = dist

Assign case-frequencies of cov_district and number of inhabitants of districts to the shapefile

In [0]:
district_cases_dict = cov_district.set_index('bkz').to_dict()['freq']

In [0]:
austria['cases'] = pd.to_numeric(austria['district_id'])

#not all bezirk ids are present in the dict, add the missing ones and assign 1 case
for i,row in austria.iterrows():
  if int(row['district_id']) not in list(district_cases_dict.keys()):
    district_cases_dict[int(row['district_id'])] = 1

In [0]:
austria['cases'] = pd.to_numeric(austria['cases'].replace(district_cases_dict))

In [0]:
district_inhabitants_dict = districts.set_index('district_id').to_dict()['inhabitants']

In [0]:
austria['inhabitants'] = austria['district_id']
austria['inhabitants'] = pd.to_numeric(austria['inhabitants'].replace(district_inhabitants_dict))

In [0]:
#plot the austria map with cases as values


Show the relative cases 

In [0]:
# calculate the relative cases, store them in the column 'relative_cases'


In [0]:
austria.head()

In [0]:
# plot the relative cases on the austria map


[Lineplot](https://matplotlib.org/3.2.1/api/_as_gen/matplotlib.pyplot.plot.html) of general data

In [0]:
general = pd.read_csv("/content/covid-data/latest/allgemein.csv", sep=";")

In [0]:
general

In [0]:
# for every folder in the archive, load the general data and add them to the dataframe
archive_folder = "/content/covid-data/archive/"
data = []
general = pd.DataFrame(columns=['erkrankungen', 'hospitalisiert', 'intensivstation', 'nr_tests', 'date'])
folders = os.listdir(archive_folder)
folders.sort()
for folder in folders:
  d = pd.read_csv(os.path.join(archive_folder, folder, "ts", "allgemein.csv"), sep=";")
  general = general.append(d)

In [0]:
general

In [0]:
# call the .plot.line() function (https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.plot.line.html)


In [0]:
# plot with subplots=True


In [0]:
# plot with only erkrankungen, hospitalisiert and intensivstation


Lineplot including healthy people

In [0]:
def load_and_clean_data(filename, aggregation_functions):
  data = pd.read_csv(filename, sep=";")

  #aggregate all the data for the single states. we are interested in the overall numbers
  data = data.groupby(data['date'], as_index=False).aggregate(aggregation_functions).reindex(columns=data.columns)
  data['date'] = data['date'].apply(lambda x : x.split()[0])
  data = data.set_index(data['date'])
  data = data.drop(columns=['nuts2', 'date'])
  data = data.loc[~data.index.duplicated(keep='first')]
  return data

In [0]:
healthy = load_and_clean_data("/content/covid-data/ts/gesundungen_bl.csv", {'nuts2': 'first', 'gesundungen': 'sum'})
hospital = load_and_clean_data("/content/covid-data/ts/hospitalisierungen_bl.csv", {'nuts2': 'first', 'hospitalisiert': 'sum', 'intensivstation': 'sum', 'nr_tests': 'first'})
dead = load_and_clean_data("/content/covid-data/ts/sterbefaelle_bl.csv", {'nuts2': 'first', 'todesfaelle': 'sum'})

In [0]:
# get the data about sick people from the general dataframe
sick = general
sick['date'] = sick['date'].apply(lambda x : x.split()[0])
sick = sick.set_index(sick['date'])
sick = sick.drop(columns=['hospitalisiert', 'intensivstation', 'nr_tests', 'date'])
sick = sick.loc[~sick.index.duplicated(keep='first')]

In [0]:
result = pd.concat([sick, healthy, hospital, dead], axis=1, sort=True)

In [0]:
result['aktuell_krank'] = result['erkrankungen'] - result['gesundungen'] - result['todesfaelle']

In [0]:
# plot only erkrankungen, gesundungen, aktuell_krank, hospitalisiert, intensivstation, todesfaelle
